In [1]:
# Fusion ART Model for Professor Data
# Python fusionART.py code by Dr. Budhitama Subagdja, modified for this dataset by Patrick Tjahjadi

# ----- Imported Libraries -----
from fusionART import *

from collections import defaultdict

import pandas as pd

import math

import random

from numpy import array
from numpy import argmax

import numpy as np

from itertools import islice

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 

import gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath
import gensim.downloader as api
from gensim import corpora

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import csv

import wikipedia

import dill
# -------------------------------

data = pd.read_csv('SCSE ProfProfile.csv', encoding='ISO-8859-1')

In [2]:
# Function to convert 1D lists into 2D, based on the number of elements per list.
def convert_to_2d(list_to_convert, list_per_entry): 
    it = iter(list_to_convert) 
    return [list(islice(it, i)) for i in list_per_entry] 

In [3]:
def search_for_ngrams(ngram_list, text_list):
    ngrammed_text = []
    for i in range(0, len(text_list)):
        bigram = text_list[i-1].lower()+"_"+text_list[i].lower()
        trigram = text_list[i-2].lower()+"_"+text_list[i-1]+"_"+text_list[i].lower()
        fourgram = text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        fivegram = text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        sixgram = text_list[i-5].lower()+"_"+text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        if bigram in ngram_list:
            ngrammed_text.append(bigram)
        if trigram in ngram_list:
            ngrammed_text.append(trigram)
        if fourgram in ngram_list:
            ngrammed_text.append(fourgram)
        if fivegram in ngram_list:
            ngrammed_text.append(fourgram)
        if sixgram in ngram_list:
            ngrammed_text.append(fourgram)
        ngrammed_text.append(text_list[i])
    return ngrammed_text

In [4]:
# Function to clean words from punctuation and remove capital case to standardise text tokens
def clean_text(word):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_123456789~'''
    no_punct = ""
    for char in word:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct.lower()

In [5]:
# Function accepts a vector and normalise its values to a range between 0 and 1
def normalise_vector(vector):
    normalised_vector = [0] * len(vector)
    max_value = max(vector)
    min_value = min(vector)
    for index in range(0, len(vector)):
        normalised_vector[index] = (vector[index] - min_value) / (max_value - min_value)
    return normalised_vector

In [6]:
# Convert each attribute of the dataset to separate lists
name = []
group = []
university = []
research_interest = []
for idx, row in data.iterrows():
    name.append(row[0])
    group.append(row[1])
    university.append(row[2])
    research_interest.append(row[3])

# Problem: research_interest and university uses semicolons to denote multiple interests and universities
# Change their format to a list of universities and research interests of a person

university_processed = []
for entry in university:
    uni = entry.split(";")
    university_processed.append(uni)

research_interest_processed = []
for entry in research_interest:
    interests = entry.split(";")
    research_interest_processed.append(interests)    

    
# Break university_processed and research_interest_processed to 1D lists for one hot encoding
university_1d = []
for universities in university_processed:
    for uni in universities:
        university_1d.append(uni)
        
research_interest_1d = []
for interests in research_interest_processed:
    for interest in interests:
        research_interest_1d.append(interest)
        


In [7]:
# Determine the number of universities and research interests per professor

universities_per_person = []
for universities in university_processed:
    universities_per_person.append(len(universities))
    

research_interest_per_person = []
for research_interests in research_interest_processed:
    research_interest_per_person.append(len(research_interests))


In [8]:
# Research interest preprocessing, to consider each research keywords as a phrase
for index in range(0, len(research_interest_1d)):
    research_interest_1d[index] = research_interest_1d[index].replace(" ", "_")
    research_interest_1d[index] = research_interest_1d[index].lower()
    research_interest_1d[index] = research_interest_1d[index].replace(":", "")
    research_interest_1d[index] = research_interest_1d[index].replace(",", "")


In [9]:
# One Hot Encoding for name, group, university and research interests
name_label_encoder = LabelEncoder()
uni_label_encoder = LabelEncoder()
group_label_encoder = LabelEncoder()
research_label_encoder = LabelEncoder()

onehot_encoder = OneHotEncoder(sparse=False)

name_integer_encoded = name_label_encoder.fit_transform(name)
name_integer_encoded = name_integer_encoded.reshape(len(name_integer_encoded), 1)
name_onehot = onehot_encoder.fit_transform(name_integer_encoded)

group_integer_encoded = group_label_encoder.fit_transform(group)
group_integer_encoded = group_integer_encoded.reshape(len(group_integer_encoded), 1)
group_onehot = onehot_encoder.fit_transform(group_integer_encoded)

university_integer_encoded = uni_label_encoder.fit_transform(array(university_1d))
university_integer_encoded = university_integer_encoded.reshape(len(university_integer_encoded), 1)
university_onehot = onehot_encoder.fit_transform(university_integer_encoded)

research_interest_integer_encoded = research_label_encoder.fit_transform(array(research_interest_1d))
research_interest_integer_encoded = research_interest_integer_encoded.reshape(len(research_interest_integer_encoded), 1)
research_interest_onehot = onehot_encoder.fit_transform(research_interest_integer_encoded)


In [10]:
# Convert the numpy arrays to lists
name_onehotlist = name_onehot.tolist()
group_onehotlist = group_onehot.tolist()
university_onehotlist = university_onehot.tolist()
research_interest_onehotlist = research_interest_onehot.tolist()


In [11]:
"""
# Perform the one-hot research keywords noise flipping here

def one_hot_keyword_flip(query_research, noise_limit):
    
    query_research = query_research.split(";")
    
    # Preprocess the research keyword queries to the format used for word embedding (phrases)
    for index in range(0, len(query_research)):
        query_research[index] = query_research[index].replace(" ", "_")
        query_research[index] = query_research[index].lower()
        query_research[index] = query_research[index].replace(":", "")
        query_research[index] = query_research[index].replace(",", "")

    # Convert all the research keywords into input vectors
    for research_index in range(0, len(query_research)):
        for onehot_vec in research_interest_onehotlist:
            if (str(query_research[research_index]).rstrip("\n") == 
                research_label_encoder.inverse_transform([argmax(onehot_vec)])[0]):
                query_research[research_index] = onehot_vec
                

        
    # Remove keywords that could not be converted to input vectors
    index = 0
    while (index < len(query_research)):
        if type(query_research[index]) != list:
            del query_research[index]
        else:
            index += 1


    # Combine all the research keywords inputs into one vector that contains all research keywords
    research_vector = [sum(research_ints) for research_ints in zip(*query_research)]
    
    # With a defined probability, flip the binary research keyword vectors due to noise
    num_change = 0
    for index in range(0, len(research_vector)):
        noise_value = random.random()
        if (noise_value < noise_limit):
            num_change += 1
            if (research_vector[index] == 0):
                research_vector[index] = 1
            else:
                research_vector[index] = 0
    print(num_change)
    return research_vector
"""

'\n# Perform the one-hot research keywords noise flipping here\n\ndef one_hot_keyword_flip(query_research, noise_limit):\n    \n    query_research = query_research.split(";")\n    \n    # Preprocess the research keyword queries to the format used for word embedding (phrases)\n    for index in range(0, len(query_research)):\n        query_research[index] = query_research[index].replace(" ", "_")\n        query_research[index] = query_research[index].lower()\n        query_research[index] = query_research[index].replace(":", "")\n        query_research[index] = query_research[index].replace(",", "")\n\n    # Convert all the research keywords into input vectors\n    for research_index in range(0, len(query_research)):\n        for onehot_vec in research_interest_onehotlist:\n            if (str(query_research[research_index]).rstrip("\n") == \n                research_label_encoder.inverse_transform([argmax(onehot_vec)])[0]):\n                query_research[research_index] = onehot_vec\n 

In [12]:
"""
# Research keywords query for 0% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0.1)
        name_keyword_dict[batch_name] = flipped_vector
"""


"\n# Research keywords query for 0% noise\nname_keyword_dict = {}\n\n# Use a batch file as input for the research keywords query\nwith open('research_query.bat', 'r') as f:\n    reader = csv.reader(f, delimiter = '\t')\n    for batch_name, keywords in reader:\n        flipped_vector = one_hot_keyword_flip(keywords, 0.1)\n        name_keyword_dict[batch_name] = flipped_vector\n"

In [13]:
"""
noisy_research_interest_1d = []
research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    for index in range(0, len(noisy_research_vector)):
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
    research_interest_per_person.append(num_res_ints)
"""

'\nnoisy_research_interest_1d = []\nresearch_interest_per_person = []\nfor professor in name:\n    num_res_ints = 0\n    noisy_research_vector = name_keyword_dict[professor]\n    for index in range(0, len(noisy_research_vector)):\n        if noisy_research_vector[index] == 1.0:\n            ind_list = [0] * len(noisy_research_vector)\n            ind_list[index] = 1\n            num_res_ints += 1\n            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]\n            noisy_research_interest_1d.append(research_name)\n    research_interest_per_person.append(num_res_ints)\n'

In [14]:
# Create a list of unique names, groups, universities and research interests to define the model schema
unique_name = []
unique_group = []
unique_university = []
unique_research_interest = []
[unique_name.append(i) for i in name if i not in unique_name]
[unique_group.append(i) for i in group if i not in unique_group]
[unique_university.append(i) for i in university_1d if i not in unique_university]
[unique_research_interest.append(i) for i in research_interest_1d if i not in unique_research_interest]
pass

In [15]:
# Create a word2vec model that contains all research keywords
research_tokens = convert_to_2d(research_interest_1d, research_interest_per_person)
research_word2vec = Word2Vec(research_tokens, min_count = 0)


In [16]:
# Gather text tokens from Wikipedia articles for all research keywords 
wiki_text = []
for keyword in research_interest_1d:
    try:
        p = wikipedia.page(keyword)
        wiki_text.append(p.content.split())
        
    # Include exception handling if the Wikipedia article can not be found
    except wikipedia.exceptions.PageError:
        pass
    except wikipedia.DisambiguationError:
        pass


From cffi callback <function _verify_callback at 0x000001B361998510>:
Traceback (most recent call last):
  File "C:\Users\patpa\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?prop=info%7Cpageprops&inprop=url&ppprop=disambiguation&redirects=&titles=Active+shape+model&format=json&action=query (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [17]:
# Clean all text from Wikipedia articles
for text_index in range(0, len(wiki_text)):
    for word_index in range(0, len(wiki_text[text_index])):
        wiki_text[text_index][word_index] = clean_text(wiki_text[text_index][word_index])
        

In [18]:
# Search for research keyword n-grams from Wikipedia articles
ngram_wiki_tokens = []

for text in wiki_text:
    ngram_wiki_tokens.append(search_for_ngrams(unique_research_interest, text))

In [19]:
# Train the word2vec model from the cleaned, n-grammed Wikipedia tokens
research_word2vec.build_vocab(ngram_wiki_tokens, update = True)
research_word2vec.train(ngram_wiki_tokens, total_examples = research_word2vec.corpus_count, epochs = research_word2vec.iter)

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(30837, 41390)

In [20]:
# Open the custom corpus text file and tokenise them. Group them with 2500 words each
testtext = []
with open('testtext.txt','r', encoding="utf-8") as f:
    for line in f:
        for word in line.split():
            testtext.append(word)  
        
test_tokens = convert_to_2d(testtext, [2500] * (math.ceil(len(testtext)/2500)))


In [21]:
# Search for research keyword n-grams from the custom corpus and clean them
ngram_test_tokens = []

for text in test_tokens:
    cleaned_text = []
    for word in text:
        cleaned_text.append(clean_text(word))
    ngram_test_tokens.append(search_for_ngrams(unique_research_interest, cleaned_text))

In [22]:
# Train the word2vec model by including the cleaned, n-grammed custom corpus tokens
research_word2vec.build_vocab(ngram_test_tokens, update = True)
research_word2vec.train(ngram_test_tokens, total_examples = research_word2vec.corpus_count, epochs = research_word2vec.iter)

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(169223, 219210)

In [23]:
# Vocabulary list for the word2vec model
list(research_word2vec.wv.vocab)

['computer_graphics',
 'shape_modelling',
 'virtual_reality',
 'web_visualization',
 'visualization_on_the_grid',
 'cybermedicine',
 'scientific_visualization',
 'high_level_synthesis',
 'application_specific_processors',
 'heterogeneous_mpsoc',
 'synthesis_for_emerging_technology',
 'distributed_systems',
 'security_and_privacy',
 'data_analytics',
 'algorithms',
 'self_organization',
 'graphs_querying_and_mining',
 'databases',
 'data_mining',
 'machine_learning',
 'real_time_systems',
 'cyber_physical_systems',
 'formal_methods',
 'ultra_wideband_radio',
 'modulation_and_multiple_access',
 'communication_algorithms',
 'digital_signal_processing',
 'multi_agent_systems',
 'game_theory',
 'optimization',
 'intelligent__e-commerce',
 'image_coding',
 'video_coding',
 'wireless_video',
 'multimedia_networking',
 'parallel_and_distributed_simulation',
 'programming_environments_and_tools',
 'parallel_algorithms_and_architectures',
 'system_performance_analysis',
 'computer_vision',
 'int

In [24]:
# Find the top 5 most similar keywords for all research keywords
for keyword in sorted(unique_research_interest):
    print("Top 5 similar words from "+keyword+": ")
    print(research_word2vec.wv.most_similar(keyword, topn = 5))
    print()

Top 5 similar words from ad_hoc_and_mobile_networks: 
[('sword', 0.3324775695800781), ('manipulating', 0.3242499828338623), ('viewer', 0.2722207009792328), ('cyber_physical_systems', 0.27097469568252563), ('partholding', 0.25012731552124023)]

Top 5 similar words from agent_oriented_software_engineering: 
[('authentication', 0.9981623888015747), ('usually', 0.9981376528739929), ('available', 0.9981184005737305), ('approach', 0.9981107711791992), ('software_engineering', 0.9980950355529785)]

Top 5 similar words from algorithms: 
[('but', 0.9999624490737915), ('we', 0.999956488609314), ('with', 0.9999548196792603), ('is', 0.9999547004699707), ('or', 0.9999539852142334)]

Top 5 similar words from algorithms_and_data_structure: 
[('way', 0.9977594614028931), ('back', 0.9977384805679321), ('makes', 0.9977350234985352), ('access', 0.9977229833602905), ('get', 0.9977197647094727)]

Top 5 similar words from animation_and_visualization: 
[('bioshock', 0.30365413427352905), ('magnavox', 0.29664

In [25]:
# Map each research keyword as a weight vector that measures its similarity with all other research keywords
research_interest_weight_dict = defaultdict(list)
for keyword1 in sorted(unique_research_interest):
    keyword1_weights = []
    for keyword2 in sorted(unique_research_interest):
        # Round to 5 decimal places for all similarity scores
        keyword1_weights.append(round(research_word2vec.wv.similarity(w1=keyword1, w2=keyword2), 5))
    research_interest_weight_dict[keyword1] = keyword1_weights
    


In [26]:
# Provide weight vectors for each research interest
research_interest_weights = []
for res_int in research_interest_1d:
    research_interest_weights.append(research_interest_weight_dict[res_int])

In [27]:
# Problem: each professor has multiple universities and research interests
# Convert these 1D arrays to 2D, to associate them to each professor

university_onehotlist_processed = []
research_interest_onehotlist_processed = []

university_onehotlist_processed = convert_to_2d(university_onehotlist, universities_per_person)
research_interest_processed = convert_to_2d(research_interest_weights, research_interest_per_person)

In [28]:
# Create an array that allows multiple universities and research interests per professor by summation
university_2d = []

for universities_per_professor in university_onehotlist_processed:
    university_2d.append([sum(x) for x in zip(*universities_per_professor)])

    
research_interest_2d = []
for research_interests_per_professor in research_interest_processed:
    research_interest_2d.append([sum(x) for x in zip(*research_interests_per_professor)])

# Since universities can be duplicate, change so that the maximum value of each university is 1
for universities_per_professor in university_2d:
    for index in range(0, len(universities_per_professor)):
        if universities_per_professor[index] > 1:
            universities_per_professor[index] = 1.0


In [29]:
# Normalise the values of all research keyword vectors to a range between 0 and 1
normalised_research_interest_2d = []
for research_vector in research_interest_2d:
    normalised_research_interest_2d.append(normalise_vector(research_vector))

In [30]:
# Define the schema for the FusionART model
model_schema = [{'name': data.columns[0], 'compl': False, 'attrib': sorted(unique_name)}, 
                {'name': data.columns[1], 'compl': False, 'attrib': sorted(unique_group)},
                {'name': data.columns[2], 'compl': False, 'attrib': sorted(unique_university)},
                {'name': data.columns[3], 'compl': False, 'attrib': sorted(unique_research_interest)}]

In [31]:
# Initialise the FusionART model
model = FusionART(schema = model_schema, beta = [1.0, 1.0, 1.0, 1.0], alpha = [0.1, 0.1, 0.1, 0.1], 
               gamma = [0.25, 0.25, 0.25, 0.25], rho = [1, 1, 1, 1])
model.F1Fields

[{'name': 'name',
  'compl': False,
  'attrib': ['AS Madhukumar',
   'Alexei Sourin',
   'Anupam Chattopadhyay',
   'Anwitaman Datta',
   'Arijit Khan',
   'Arvind Easwaran',
   'Bo An',
   'Cai Jianfei',
   'Cai Wentong',
   'Cham Tat Jen',
   'Chan Syin',
   'Chia Liang Tien Clement',
   'Chng Eng Siong',
   'Deepu Rajan',
   'Deng Ruilong',
   'Douglas Leslie Maskell',
   'Dusit Niyato',
   'Eric Cambria',
   'Gao Cong',
   'Goh Wooi Boon',
   'Guan Cuntai',
   'He Ying',
   'Huang Shell Ying',
   'Hui Siu Cheung',
   'Jagath C. Rajapakse',
   'Ke Yiping Kelly',
   'Kong Wai Kin Adams',
   'Kwoh Chee Keong',
   'Lam Kwok Yan',
   'Lam Siew Kei',
   'Lau Chiew Tong',
   'Lee Bu Sung Francis',
   'Li Fang Flora',
   'Li Mo',
   'Li Yi',
   'Liang Qianhui Althea',
   'Lin Feng',
   'Lin Guosheng',
   'Lin Shang Wei',
   'Lin Weisi',
   'Liu Weichen',
   'Liu Yang',
   'Loke Yuan Ren',
   'Long Cheng',
   'Loy Chen Change (Cavan)',
   'Lu Shijian',
   'Luo Jun',
   'Mahardhika Pratama',

In [32]:
# Store the data to the Fusion ART model

for i in range(0, len(name)):
    model.updateF1bySchema([{'name': data.columns[0], 'val': name_onehotlist[i]}, 
                            {'name': data.columns[1], 'val': group_onehotlist[i]}, 
                            {'name': data.columns[2], 'val': university_2d[i]}, 
                            {'name': data.columns[3], 'val': normalised_research_interest_2d[i]}])

    
    print("resonance search: ")
    J = model.resSearch()
    print("selected ", J)
    if model.uncommitted(J):
        print ('uncommitted')

    model.autoLearn(J)
    model.doReadoutAllFields(J)
                    
    
model.displayNetwork()

resonance search: 
selected  0
uncommitted
resonance search: 
selected  1
uncommitted
resonance search: 
selected  2
uncommitted
resonance search: 
selected  3
uncommitted
resonance search: 
selected  4
uncommitted
resonance search: 
selected  5
uncommitted
resonance search: 
selected  6
uncommitted
resonance search: 
selected  7
uncommitted
resonance search: 
selected  8
uncommitted
resonance search: 
selected  9
uncommitted
resonance search: 
selected  10
uncommitted
resonance search: 
selected  11
uncommitted
resonance search: 
selected  12
uncommitted
resonance search: 
selected  13
uncommitted
resonance search: 
selected  14
uncommitted
resonance search: 
selected  15
uncommitted
resonance search: 
selected  16
uncommitted
resonance search: 
selected  17
uncommitted
resonance search: 
selected  18
uncommitted
resonance search: 
selected  19
uncommitted
resonance search: 
selected  20
uncommitted
resonance search: 
selected  21
uncommitted
resonance search: 
selected  22
uncommitte

In [38]:
# Perform the research keywords query here

def query_by_research_with_noise(batch_name, query_research, output_file, noise_limit):
    
    query_research = query_research.split(";")
    output_file.write("Research Keywords: ")
    output_file.write(', '.join(query_research)+"\n")
    
    
    # Preprocess the research keyword queries to the format used for word embedding (phrases)
    for index in range(0, len(query_research)):
        query_research[index] = query_research[index].replace(" ", "_")
        query_research[index] = query_research[index].lower()
        query_research[index] = query_research[index].replace(":", "")
        query_research[index] = query_research[index].replace(",", "")
    
    # Convert all the research keywords into input vectors
    for research_index in range(0, len(query_research)):
        for onehot_vec in research_interest_onehotlist:
            if (str(query_research[research_index]).rstrip("\n") == 
                research_label_encoder.inverse_transform([argmax(onehot_vec)])[0]):
                query_research[research_index] = onehot_vec
                
    # Remove keywords that could not be converted to input vectors
    index = 0
    while (index < len(query_research)):
        if type(query_research[index]) != list:
            del query_research[index]
        else:
            index += 1
    
    # Combine all the research keywords inputs into one vector that contains all research keywords
    research_vector = [sum(research_ints) for research_ints in zip(*query_research)]
    
    # With a defined probability, flip the binary research keyword vectors due to noise
    for index in range(0, len(research_vector)):
        noise_value = random.random()
        if (noise_value < noise_limit):
            research_vector[index] = 1 - research_vector[index]
            
    model.setParam('gamma', [0,0,0,1])
    model.setParam('rho', [0,0,0,0])
    model.updateF1bySchema([{'name': data.columns[3], 'val': research_vector}])
    model.compChoice()

    # Find the nodes that have the highest F2 value
    maxF2value = model.codes[0]['F2']
    maxF2indexes = []

    for i in range(0, len(model.codes)-1):
        F2value = model.codes[i]['F2']
        if (F2value > maxF2value):
            maxF2value = F2value
            maxF2indexes = [i]
        elif (F2value == maxF2value):
            maxF2indexes.append(i)
            
    print(maxF2indexes)
    
    # Retrieve the data of the node(s) that have the highest F2 value
    output_file.write("Professor that follows these research keywords:")    
    for node in maxF2indexes:
        name_vector = model.codes[node]['weights'][0]
        for index in range(0, len(name_vector)):
            # Iterate through the vector and find the indexes that has a value of 1
            if (name_vector[index] == 1):
                # Retrieve the data from the model's 'attrib' attribute in F1 
                retrieved_name = model.F1Fields[0]['attrib'][index]
                output_file.write(retrieved_name)
    
    output_file.write("\n\n")
    return(retrieved_name)


In [39]:
# Research keywords query for 0% noise
output_file = open("output_zero_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(batch_name, keywords, output_file, 0)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()


[0]
[1]
[2]
[44]
[4]
[5]
[6]
[7]
[8]
[9]
[10]


KeyboardInterrupt: 

In [40]:
# Research keywords query for 10% noise
output_file = open("output_zero_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(batch_name, keywords, output_file, 0.1)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()


[72]
[55]
[8]
[49]
[63]
[5]
[38]
[70]
[8]
[73]
[63]
[11]
[33]
[11]
[69]
[29]
[44]
[66]
[18]
[50]
[56]
[33]
[70]
[23]
[29]
[49]
[72]
[33]
[14]
[56]
[50]
[12]
[74]
[33]
[34]
[39]
[71]
[26]
[38]
[19]
[56]
[9]
[4]
[56]
[44]
[4]
[11]
[47]
[26]
[49]
[57]
[23]
[29]
[56]
[69]
[11]
[56]
[10]
[44]
[59]
[60]
[14]
[62]
[63]
[64]
[36]
[69]
[70]
[69]
[33]
[70]
[33]
[72]
[77]
[74]
[17]
[62]
[56]
[19]
[50]
[29]
[22]
[69]
